### **6.1 - The (not so) dynamic tasks**

¿Qué pasa si tienes el siguiente caso de uso en el que quieres extraer datos de diferentes partners, por ejemplo, el partner A, B y C. La cosa es que la forma de extraer datos de tus diferentes partners es realmente similar. ¿Tienes que implementar la misma tarea para cada uno de tus partners? ¿Qué pasa si tienes como 50 partners diferentes? Obviamente, en algún momento podría ser realmente difícil hacer eso y absolutamente no es adecuado ya que será difícil mantener esas tareas. Por lo tanto, una manera es crear esas tareas dinámicamente y que sean realmente similares entre sí y sólo poner las pocas diferencias en el diccionario, por ejemplo. Así que aquí tendrías un bucle encargado de crear esas tareas dinámicamente, y entonces terminarás con la tarea, extract A, luego extract B y extract C. Pero no vas a implementar cada tarea una por una. Esas tareas se generarán dinámicamente en base al diccionario. 

<center><img src="https://i.postimg.cc/Dy1Sn3Zc/a1312.png"></center>

Vamos a ver cómo hacer esto. Así que vamos a aplicar este ejemplo muy simple en el siguiente data pipeline. Así que, en primer lugar, es necesario definir ese diccionario, que contiene los diferentes valores de acuerdo con el partner con el que se quiere tratar. En ese caso, vamos a crear un diccionario "partners":

<center><img src="https://i.postimg.cc/7ZDfLtJr/a1313.png"></center>

Así que tienes los diferentes partners en tu diccionario está listo. El siguiente paso es definir el bucle encargado de crear las diferentes tareas "extract" correspondientes a "snowflake", "netflix" y "astronomer". Así que, en la parte inferior de tu Dag, crea ese bucle, así que escribe:

<center><img src="https://i.postimg.cc/cLm6t7fg/a1314.png"></center>

Luego, obviamente, no olvides añadir esos argumentos a la tarea "extract" y puedes eliminar esas dos variables:

<center><img src="https://i.postimg.cc/LsQXNHCm/a1315.png"></center>

A continuación, elimina algunas líneas, esta:

<center><img src="https://i.postimg.cc/G2BpfZVG/a1316.png"></center>

Así como esta otra:

<center><img src="https://i.postimg.cc/SKxR6XGZ/a1317.png"></center>

Y quieres llamar a "process_tasks" con el argumento "extract( )". Si haces eso, terminarás con un error. ¿Por qué? porque esta función "process_tasks" devuelve un taskgroup, pero ese taskgroup mantiene el mismo group ID 

<center><img src="https://i.postimg.cc/gj82z3Y0/a1318.png"></center>

y no puedes tener varios taskgroups con el mismo group ID. Una cosa que puedes hacer es añadir el siguiente argumento "add_suffix_on_collision" para que Airflow añada automáticamente un sufijo para cada uno de sus group ID. 

<center><img src="https://i.postimg.cc/y6ZdyCFz/a1319.png"></center>

Si guardas el archivo y vas a la interfaz de Airflow, obtienes el siguiente data pipeline, que está muy bien porque tienes tus diferentes tareas, "extract_partners" uno y dos, pero no es perfecto, ¿verdad? Quiero decir, usted no sabe que "extract_partners" es para snowflake, entonces el segundo es para netflix y así sucesivamente. Entonces, ¿se puede mejorar ese data pipeline? Bueno, por supuesto que puedes.

<center><img src="https://i.postimg.cc/SswKbgb5/a1320.png"></center>

Así que vamos a volver a tu editor de código y una cosa que puedes hacer es cortar esta tarea

<center><img src="https://i.postimg.cc/X76v218B/a1321.png"></center>

y ponerla dentro de tu "bucle for" y luego para el task ID quieres usar partner como se define en tu diccionario:

<center><img src="https://i.postimg.cc/Qxgdvg44/a1322.png"></center>

Entonces, por último, pero no menos importante, vamos a añadir una tarea ficticia como el punto de partida de tu data pipeline, así que vamos a escribir "from airflow.operators.dummy import DummyOperator" 

<center><img src="https://i.postimg.cc/CxFx0Z9F/a1323.png"></center>

y aquí creamos una nueva tarea:

<center><img src="https://i.postimg.cc/XYQ7Yj09/a1324.png"></center>

y finalmente creas las dependencias:

<center><img src="https://i.postimg.cc/cJzxfFCH/a1325.png"></center>

Sé que puede parecer un poco difícil de entender al principio, pero básicamente aquí no olvides que "extracted_values" corresponde a "XCom args" y así puedes definir las dependencias como antes y también pasando "extracted_values" como un argumento de "process_tasks" Airflow creará automáticamente las dependencias para ti. Así que ahora, si guardas el archivo y echas un vistazo a la interfaz de usuario de Airflow, obtendrás el siguiente data pipeline, que se ve mucho mejor que antes. 

<center><img src="https://i.postimg.cc/Gpt3BWcV/a1326.png"></center>
<center><img src="https://i.postimg.cc/QtWXmcKq/a1327.png"></center>

Ahora aquí está la cosa, tienes que recordar que la creación de tareas de forma dinámica es posible sólo si los valores ya se conocen. Airflow necesita conocer de antemano la estructura de su Dag, lo que significa que no es capaz de crear tareas dinámicas basadas en la salida de una tarea. Puede crear tareas dinámicas basadas en un diccionario o en una variable, o incluso en algunas conexiones que tenga en su base de datos, pero todos esos valores ya son conocidos. Si tratas de crear tareas dinámicas basadas en la salida de una tarea, por ejemplo, el XCom o una SQL request, eso no funcionará, eso cambiará en el futuro, pero ahora mismo es así como funciona.